In [ ]:
from hdbcli import dbapi
import pandas as pd
import nltk as nltk
import seaborn as sns
from nltk.corpus import stopwords
import re

In [ ]:
connection = dbapi.connect('52.18.25.218', 39015, 'SYSTEM', 'Glorp2018!')
connection.isconnected()

In [ ]:
cursor = connection.cursor()

# Teil 2 mit SQL
## Aufgabe 1

In [ ]:
# nouns per document
cursor.execute('select ID, count(*) from "$TA_T3NTEXTIND" where TA_TYPE=\'noun\' group by ID order by count(*) desc')
nouns_list = cursor.fetchall()
nouns_df = pd.DataFrame(nouns_list)
nouns_df.columns = ['doc', 'count']
print(nouns_df)

ax = sns.distplot(nouns_df['count'])
ax.set(xlabel='#nouns per document')
ax.set_title('Nouns per document')
fig = ax.get_figure()

In [ ]:
# number of individual nouns per document
# todo

## Aufgabe 2

In [ ]:
# size lexica
cursor.execute('select count(distinct TA_TOKEN) from "$TA_T3NTEXTIND"')
nouns_list = cursor.fetchall()
lexica_size = pd.DataFrame(nouns_list)
lexica_size.columns = ['size']
lexica_size

In [ ]:
# size lexica without ","
cursor.execute('select count(distinct TA_TOKEN) from "$TA_T3NTEXTIND" where TA_TOKEN <> \',\'')
nouns_list = cursor.fetchall()
lexica_size = pd.DataFrame(nouns_list)
lexica_size.columns = ['size']
lexica_size

## Aufgabe 3

In [ ]:
# word frequency
cursor.execute('select TA_TOKEN, count(*) from "$TA_T3NTEXTIND" group by TA_TOKEN order by count(*) desc')
words_list = cursor.fetchall()
words_df = pd.DataFrame(words_list)
words_df.columns = ['word', 'count']
words_df

In [ ]:
# restriction to POS-Tags, categories or documents

In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
tmpDict = {}
cursor.execute('SELECT top 20 TA_TOKEN, count(*) from "$TA_T3NTEXTIND" where TA_TYPE=\'noun\' group by TA_TOKEN order by count(*) desc')
for row in cursor: 
    tmpDict[row[0]]=row[1]
wordcloud = WordCloud(width=480, height=480, margin=0).generate_from_frequencies(tmpDict) 
               # Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

## Aufgabe 4

## Aufgabe 5